# Create assistant with access to files and a function

This assistant is supposed to answer questions about the HR policy on Extraordinary leave. It has access to the following files:

- "Extraordinary leave.pdf"

There is also a function that allows an employee to request a raise.

**Note**: in the OpenAI Assistants API (in general, available at OpenAI today in beta as well), you can upload files and enable the **retrieval** tool like so:

```python
assistant = client.beta.assistants.create(
  instructions="You are a customer support chatbot. Use your knowledge base to best respond to customer queries.",
  model="gpt-4-turbo-preview",
  tools=[{"type": "retrieval"}]
)
```

You would also upload files and pass the file ids here but that is not shown.

Currently though (Feb 2024), the Azure OpenAI Assistants API does not allow you to configure the `retrieval` tool. You can only enable the `code_interpreter` tool. The `code_interpreter` tool might be able to open your documents and read them, but it's not as powerful as the retrieval tool. It basically has to use Python code to read the file and then answer questions about it.

Instead of using the `retrieval` tool, we will use an in memory vector store instead with the help of Chroma, LangChain and Azure OpenAI Embeddings.

You will need the following packages:
- openai
- langchain
- chroma
- dotenv
- PyPDF2

In [1]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

# Load environment variables from .env file
# AZURE_OPENAI_API_KEY
# AZURE_OPENAI_ENDPOINT
# AZURE_OPENAI_API_VERSION
load_dotenv()

# Create Azure OpenAI client
client = AzureOpenAI(
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    api_version=os.getenv('AZURE_OPENAI_API_VERSION')
)



## Use Chroma in-memory vector store

Below we read a PDF and convert it to text. The text is split into smaller pieces and then handed to Croma to store these pieces + a vector + metadata (file, page number)

We then do a similarity search to test if we get results from Chroma.

In [8]:
# in memory vector store with chroma (pip install chroma)
# you also need langchain for this
# you can ignore the warnings coming from this code
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

pdf = PyPDFLoader("./Extraordinary leave.pdf").load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
documents = text_splitter.split_documents(pdf)
print(documents)
print(len(documents))
db = Chroma.from_documents(documents, AzureOpenAIEmbeddings(client=client, model="text-embedding-ada-002", api_version="2023-05-15"))

# query the vector store
query = "Is extraordinary leave transferable?"
docs = db.similarity_search(query, k=3)
print(docs)
print(len(docs))

[Document(page_content='For additional information contact HR: CTG.HR@comtradegaming.comHow to report \nan extraordinary leave?\nExtraordinary leave represents absence due to infrequent events such as:\n• employee’s or their child’s wedding,\n• birth of a child,\n• death of a close family member, \n• employee’s or their family’s move,\n• blood donor session.\nIn case one of those events happen, you should inform your Project manager and Team \nmanager and register your absence in Daily report. In some instances *, HR department will \nneed adequate evidence for approvement of your request. This kind of absence must be used at the time of the event and is not transferable. You can ﬁnd detailed explanation in the relevant rule book.', metadata={'source': './Extraordinary leave.pdf', 'page': 0}), Document(page_content='*Note: Blood donor session, natural disaster, move, or in case the employee must perform their civic duties, if they are summoned to appear in court or other public authori

## Helper function to search the HR policy

The helper function does a similarity search and returns the 3 most relavant pieces on content as a JSON string. The JSON string will be the tool output that the model uses to answer a question like: "What is the policy on vacation days?".

In [9]:
import json

# function to retrieve HR questions
def hr_query(query):
    docs = db.similarity_search(query, k=3)
    docs_dict = [doc.__dict__ for doc in docs]
    return json.dumps(docs_dict)

# try the function; docs array as JSON
print(hr_query("Is extraordinary leave transferable?"))

[{"page_content": "For additional information contact HR: CTG.HR@comtradegaming.comHow to report \nan extraordinary leave?\nExtraordinary leave represents absence due to infrequent events such as:\n\u2022 employee\u2019s or their child\u2019s wedding,\n\u2022 birth of a child,\n\u2022 death of a close family member, \n\u2022 employee\u2019s or their family\u2019s move,\n\u2022 blood donor session.\nIn case one of those events happen, you should inform your Project manager and Team \nmanager and register your absence in Daily report. In some instances *, HR department will \nneed adequate evidence for approvement of your request. This kind of absence must be used at the time of the event and is not transferable. You can \ufb01nd detailed explanation in the relevant rule book.", "metadata": {"page": 0, "source": "./Extraordinary leave.pdf"}, "type": "Document"}, {"page_content": "*Note: Blood donor session, natural disaster, move, or in case the employee must perform their civic duties, 

## Create the assistant with code

Here you see how to add functions to an assistant from code. It's an array of JSON that defines custom functions or built-in tools like `code_interpreter`.

We show how to upload files and add them to the assistant. As discussed earlier, we cannot enable the built-in `retrieval` tool yet.

In [10]:


# upload the HR policy and get the file id
# this would be used if retrieval was supported
# just shown here for completeness
hr_policy_file = client.files.create(
    file=open('Extraordinary leave.pdf', 'rb'),
    purpose='assistants'
)

# load assistant ID from assistant_id.txt
# delete that file before running this code to create a new assistant
try:
    with open('assistant_id.txt', 'r') as f:
        assistant_id = f.read()
except FileNotFoundError:
    print("The file 'assistant_id.txt' does not exist.")
    assistant_id = None

print('Assistant ID:', assistant_id)

if assistant_id is None:
    # Create a new assistant
    assistant = client.beta.assistants.create(
        name="HR Assistant",
        instructions="""You are an HR Assistant for the company Innovatek.
        You answer questions about HR policy from the the hr_query function you have access to. hr_query only answers
        questions realted to the HR policy of Innovatek. When you get an answer, check the metadata and return document paths or urls
        that were used as the source of the answer.
        Users can ask for a raise. If they do, so call the function `request_raise` with the amount they are asking for.
        If you do not know the employee before asking a raise, ask the user for their name.
        """,
        tools=[{ 
            "function": {
                "name": "request_raise",
                "description": "Request a raise for an employee",
                "parameters": {
                "type": "object",
                "properties": {
                    "employee": {
                        "type": "string",
                        "description": "Name of the employee. Ask the user if you do not know."
                        },
                    "amount": {
                        "type": "integer",
                        "description": "The raise amount, not the new salary"
                        }
                },
                "required": [
                    "employee",
                    "amount"
                ]
                }
            },
            "type": "function"
            },
            { 
            "function": {
                "name": "hr_query",
                "description": "Can answer HR related questions",
                "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "HR-related question for company Innovatek"
                        }
                },
                "required": [
                    "query"
                ]
                }
            },
            "type": "function"
            }, 
            {
                "type": "code_interpreter",  # should be set to retrieval but that is not supported yet; required or file_ids will throw error
            }
        ],
        model="gpt-4-preview", # ensure you have a deployment in the region you are using
        file_ids=[hr_policy_file.id] # pass the file_ids, max 20
    )

    # write the assistant ID to a file
    assistant_id = assistant.id
    with open('assistant_id.txt', 'w') as f:
        f.write(assistant_id)
    print('Assistant created:', assistant_id)
    

The file 'assistant_id.txt' does not exist.
Assistant ID: None


BadRequestError: Error code: 400 - {'error': {'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_found'}}

## Create a thread and add a message

Here we add a message with a question about the HR policy.

Instead of using the retrieval tool, we hope the model will use the hr_query function to answer the question.

In [87]:
# Create a thread
thread = client.beta.threads.create()

# Threads have an id as well
print("Thread id: ", thread.id)

Thread id:  thread_YQTew8F0sOViu0w6mVaCZVpW


In [88]:

# you can come back to this cell to ask more questions in the thread
# simply change the message and run the cell
# to start a new thread, run the cell above

import time
from IPython.display import clear_output

# function returns the run when status is no longer queued or in_progress
def wait_for_run(run, thread_id):
    while run.status == 'queued' or run.status == 'in_progress':
        run = client.beta.threads.runs.retrieve(
                thread_id=thread_id,
                run_id=run.id
        )
        time.sleep(0.5)

    return run


# create a message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What company cars can I get?"
)

# create a run 
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id, # use the assistant id defined in the first cell
    # tools=[{"type": "retrieval"}] # using the retrieval tool is not supported at run level either
)

# wait for the run to complete
run = wait_for_run(run, thread.id)

# show information about the run
# should indicate that run status is requires_action
# should contain information about the tools to call
print(run.model_dump_json(indent=2))

{
  "id": "run_HnZlvhxWZGk7iUtQuLvMwhSY",
  "assistant_id": "asst_Ol9RnLsiDCD6khGtF05LarPm",
  "cancelled_at": null,
  "completed_at": null,
  "created_at": 1707573142,
  "expires_at": 1707573742,
  "failed_at": null,
  "file_ids": [
    "assistant-91XlM4DmZkSrpJRrM0cxcfNJ"
  ],
  "instructions": "You are an HR Assistant for the company Innovatek.\n        You answer questions about HR policy from the the query_hr function you have access to. query_hr only answers\n        questions realted to the HR policy of Innovatek. When you get an answer, check the metadata and return document paths or urls\n        that were used as the source of the answer.\n        Users can ask for a raise. If they do, so call the function `request_raise` with the amount they are asking for.\n        If you do not know the employee before asking a raise, ask the user for their name.\n        ",
  "last_error": null,
  "metadata": {},
  "model": "gpt-4-preview",
  "object": "thread.run",
  "required_action": {

## Check if we need to run a function

Note that when you ask for a raise, the model will ask for your name. Use the cell where you add a message to add your name and run that cell and the next ones.

In [89]:
if run.required_action:
    # get tool calls and print them
    # check the output to see what tools_calls contains
    tool_calls = run.required_action.submit_tool_outputs.tool_calls
    print("Tool calls:", tool_calls)

    # we might need to call multiple tools
    # the assistant API supports parallel tool calls
    # we account for this here although we only have one tool call
    tool_outputs = []
    for tool_call in tool_calls:
        func_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)

        # call the function with the arguments provided by the assistant
        if func_name == "hr_query":
            result = hr_query(**arguments)
        elif func_name == "request_raise":
            result = "Request sumbitted. It will take two weeks to review."

        # append the results to the tool_outputs list
        # you need to specify the tool_call_id so the assistant knows which tool call the output belongs to
        tool_outputs.append({
            "tool_call_id": tool_call.id,
            "output": json.dumps(result)
        })

    # now that we have the tool call outputs, pass them to the assistant
    run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=tool_outputs
    )

    print("Tool outputs submitted")

    # now we wait for the run again
    run = wait_for_run(run, thread.id)
else:
    print("No tool calls identified\n")

Tool calls: [RequiredActionFunctionToolCall(id='call_wCZWDXsUkC0dDBGAGhkRZC1u', function=Function(arguments='{"query":"What company cars are available for employees?"}', name='hr_query'), type='function')]
Tool outputs submitted


## Pretty print the thread messages

In reverse order to start from the first question.

In [90]:
import json

messages = client.beta.threads.messages.list(thread_id=thread.id)
messages_json = json.loads(messages.model_dump_json())

def role_icon(role):
    if role == "user":
        return "👤"
    elif role == "assistant":
        return "🤖"

for item in reversed(messages_json['data']):
    # Check the content array
    for content in reversed(item['content']):
        # If there is text in the content array, print it
        if 'text' in content:
            print(role_icon(item["role"]),content['text']['value'], "\n")
        # If there is an image_file in the content, print the file_id
        if 'image_file' in content:
            print("Image ID:" , content['image_file']['file_id'], "\n")

👤 What company cars can I get? 

🤖 The company cars available for Innovatek employees are:

- Tesla Model 3
- Tesla Model S
- BMW i4
- BMW i5

Gasoline or diesel cars are not allowed as company cars.

The source of this information is found in the Innovatek HR Policy document. The relevant document path is `./Innovatek.pdf`. 

